In [ ]:
import pandas as pd
import pickle
from google.colab import drive
drive.mount('/content/drive')
import torch
import torchaudio
import os
from torch.nn.functional import normalize
import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np


In [ ]:

pickle_in_y = open("/content/drive/MyDrive/1_new/law/all_y_train.pickle","rb")
y_train = pickle.load(pickle_in_y)
pickle_in_y.close()
pickle_in_y1 = open("/content/drive/MyDrive/1_new/law/all_y_test.pickle","rb")
y_test = pickle.load(pickle_in_y1)
pickle_in_y1.close()

In [ ]:
x_train_z = []
x_test_z = []

x_to_add = open("/content/drive/MyDrive/1_new/law/all_x_train.pickle","rb")
x_train = pickle.load(x_to_add)
x_to_add.close()

x_to_add1 = open("/content/drive/MyDrive/1_new/law/all_x_test.pickle","rb")
x_test = pickle.load(x_to_add1)
x_to_add1.close()

  


In [ ]:
 
my_tensor = torch.cat(x_train)
my_tensor = torch.unsqueeze(my_tensor, 1)
print(my_tensor.shape)


norm_x_test = torch.cat(x_test)
tensor_test = torch.unsqueeze(norm_x_test, 1)
tensor_test.shape

torch.Size([11950, 1, 149, 32])


torch.Size([2957, 1, 149, 32])

In [ ]:
temp_dickt = {'eu': 0, 'de': 1, 'es': 2, 'ca': 3, 'fr': 4, 'zh-CN': 5, 'en': 6}
# for i, x in enumerate(set(y_test)):
#   temp_dickt[x] = i 
# {'eu': 0, 'de': 1, 'es': 2, 'ca': 3, 'fr': 4, 'zh-CN': 5, 'en': 6}
y_train_num = torch.FloatTensor([temp_dickt[x] for x in y_train])
y_test_num = torch.FloatTensor([temp_dickt[x] for x in y_test])
print(temp_dickt)

{'eu': 0, 'de': 1, 'es': 2, 'ca': 3, 'fr': 4, 'zh-CN': 5, 'en': 6}


In [ ]:
pd.DataFrame(y_test_num).value_counts()

5.0    624
0.0    488
3.0    420
6.0    409
2.0    377
1.0    345
4.0    294
dtype: int64

In [ ]:
from torch import nn
import torch

DROP_OUT = 0.3



class Convolutional_Speaker_Identification(nn.Module):

    def __init__(self):

        super().__init__()

        self.conv_2d_1 = nn.Conv2d(1, 96, kernel_size=(3, 3), stride=(1, 1), padding=1)
        self.bn_1 = nn.BatchNorm2d(96)
        self.max_pool_2d_1 = nn.MaxPool2d(kernel_size=(3, 3), stride=(1, 1))

        self.conv_2d_2 = nn.Conv2d(96, 256, kernel_size=(3, 3), stride=(2, 2), padding=1)
        self.bn_2 = nn.BatchNorm2d(256)
        self.max_pool_2d_2 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))

        self.conv_2d_3 = nn.Conv2d(256, 384, kernel_size=(3, 3), padding=1)
        self.bn_3 = nn.BatchNorm2d(384)

        self.conv_2d_4 = nn.Conv2d(384, 256, kernel_size=(3, 3), padding=1)
        self.bn_4 = nn.BatchNorm2d(256)

        self.conv_2d_5 = nn.Conv2d(256, 256, kernel_size=(3, 3), padding=1)
        self.bn_5 = nn.BatchNorm2d(256)
        self.max_pool_2d_3 = nn.MaxPool2d(kernel_size=(5, 3), stride=(3, 2))

        self.conv_2d_6 = nn.Conv2d(256, 4096, kernel_size=(9, 1), padding=0)
        self.drop_1 = nn.Dropout(p=DROP_OUT)

        self.global_avg_pooling_2d = nn.AdaptiveAvgPool2d((1, 1))
        self.dense_1 = nn.Linear(4096, 1024)
        self.drop_2 = nn.Dropout(p=DROP_OUT)

        self.dense_2 = nn.Linear(1024, 7)

    def forward(self, X):

        x = nn.ReLU()(self.conv_2d_1(X))
        x = self.bn_1(x)
        x = self.max_pool_2d_1(x)

        x = nn.ReLU()(self.conv_2d_2(x))
        x = self.bn_2(x)
        x = self.max_pool_2d_2(x)

        x = nn.ReLU()(self.conv_2d_3(x))
        x = self.bn_3(x)

        x = nn.ReLU()(self.conv_2d_4(x))
        x = self.bn_4(x)

        x = nn.ReLU()(self.conv_2d_5(x))
        x = self.bn_5(x)
        x = self.max_pool_2d_3(x)

        x = nn.ReLU()(self.conv_2d_6(x))
        x = self.drop_1(x)
        x = self.global_avg_pooling_2d(x)

        x = x.view(-1, x.shape[1])  # output channel for flatten before entering the dense layer
        x = nn.ReLU()(self.dense_1(x))
        x = self.drop_2(x)

        x = self.dense_2(x)
        y = nn.LogSoftmax(dim=1)(x)   # consider using Log-Softmax

        return y

    def get_epochs(self):
        return 30

    def get_learning_rate(self):
        return 0.000005

    def get_batch_size(self):
        return 32

    def to_string(self):
        return "Convolutional_Speaker_Identification_Log_Softmax_Model-epoch_"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Convolutional_Speaker_Identification().to(device)


weights = torch.FloatTensor(list(100.0 / pd.DataFrame(y_train).value_counts())).to(device)
learning_rate = model.get_learning_rate()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


criterion = torch.nn.CrossEntropyLoss(weight= weights)
epoch, batch_size = model.get_epochs(), model.get_batch_size()

sample_size = y_train_num.shape[0]

batchs = math.ceil(sample_size/batch_size)
model.train()
for e in range(epoch):
  
  for i in range(batchs):
    min = i*batch_size
    max = i*batch_size + batch_size

    if i*batch_size + batch_size > sample_size:
      max = sample_size
    
    X = my_tensor[min:max].to(device)
    y = y_train_num[min:max].long().to(device)

    model.zero_grad()

    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    train_acc = torch.sum(torch.argmax(output, dim = 1) == y)

  print(str(e)+")" + str(loss))

  if e%3 == 0:
    model.eval()
    y_p = np.argmax(model(tensor_test[:int(len(tensor_test)*0.1)].to(device)).to("cpu").detach().numpy(), axis =1)
    y_true = np.array(y_test_num[:int(len(tensor_test)*0.1)], dtype= int)

    acc = np.mean(np.array(y_p == y_true, dtype= int))
    print("Test accuracy score:", round(acc*100,2) , "%")
    if round(acc*100,2) >80.0:
      break;
    model.train()









0)tensor(1.2566, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy score: 36.95 %
1)tensor(0.9690, device='cuda:0', grad_fn=<NllLossBackward0>)
2)tensor(0.7625, device='cuda:0', grad_fn=<NllLossBackward0>)
3)tensor(0.7406, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy score: 68.14 %
4)tensor(0.6270, device='cuda:0', grad_fn=<NllLossBackward0>)
5)tensor(0.4381, device='cuda:0', grad_fn=<NllLossBackward0>)
6)tensor(0.4081, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy score: 73.22 %
7)tensor(0.2808, device='cuda:0', grad_fn=<NllLossBackward0>)
8)tensor(0.2994, device='cuda:0', grad_fn=<NllLossBackward0>)
9)tensor(0.1528, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy score: 77.63 %
10)tensor(0.1421, device='cuda:0', grad_fn=<NllLossBackward0>)
11)tensor(0.0955, device='cuda:0', grad_fn=<NllLossBackward0>)
12)tensor(0.0777, device='cuda:0', grad_fn=<NllLossBackward0>)
Test accuracy score: 76.61 %
13)tensor(0.0411, device='cuda:0', grad_fn=<Nl

In [ ]:
# torch.save(model.state_dict(), "/content/drive/MyDrive/voice/"+llen+"stat1.pth")
torch.save(model, "/content/drive/MyDrive/1_new/model/enmodel0.pth")